In [1]:
import string

parse_hex = lambda s: int(s.translate({ord(c): None for c in string.whitespace}), 16)

# 문제 해석
문제에 나온 값들이 무슨 의미를 가지는지 알아보자.

유한체 $\mathbb{F}_{p^2}$과 starting curve $E: y^2 = x^3 + 6x^2 + x$를 구성하면 다음과 같다.

In [2]:
p = parse_hex("0002341F 27177344 6CFC5FD6 81C52056 7BC65C78 3158AEA3 FDC1767A E2FFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF")
F.<x> = GF(p)[]
k.<i> = GF((p, 2), modulus=x^2 + 1)
print(k)
E = EllipticCurve(k, [0, 6, 0, 1, 0])
print(E)

Finite Field in i of size 24439423661345221551909145011457493619085780243761596511325807336205221239331976725970216671828618445898719026692884939342314733567^2
Elliptic Curve defined by y^2 = x^3 + 6*x^2 + x over Finite Field in i of size 24439423661345221551909145011457493619085780243761596511325807336205221239331976725970216671828618445898719026692884939342314733567^2


문제에 나온 값과 SIKE 표준에 있는 값 사이에는 다음 관계가 성립한다.

| 문제                           | SIKE                     |
|--------------------------------|--------------------------|
| $P_A$                          | $P_2$                    |
| $Q_A$                          | $Q_2$                    |
| $P_A-Q_A$ (x, y좌표 모두 있음) | $R_2$ (x좌표만 나와있음) |
| $P_B$                          | $P_3$                    |
| $Q_B$                          | $Q_3$                    |
| $P_B-Q_B$ (x, y좌표 모두 있음) | $R_3$ (x좌표만 나와있음) |

SIKE 표준에 있는 값을 가져오자.

In [3]:
P_A = E(parse_hex("00003CCF C5E1F050 030363E6 920A0F7A 4C6C71E6 3DE63A0E 6475AF62 1995705F 7C84500C B2BB61E9 50E19EAB 8661D25C 4A50ED27 9646CB48") +
        parse_hex("0001AD1C 1CAE7840 EDDA6D8A 924520F6 0E573D3B 9DFAC6D1 89941CB2 2326D284 A8816CC4 249410FE 80D68047 D823C97D 705246F8 69E3EA50") * i,
        parse_hex("0001AB06 6B849495 82E3F666 88452B92 55E72A01 7C45B148 D719D9A6 3CDB7BE6 F48C812E 33B68161 D5AB3A0A 36906F04 A6A6957E 6F4FB2E0") +
        parse_hex("0000FD87 F67EA576 CE97FF65 BF9F4F76 88C4C752 DCE9F8BD 2B36AD66 E04249AA F8337C01 E6E4E1A8 44267BA1 A1887B43 3729E1DD 90C7DD2F") * i)
Q_A = E(parse_hex("0000C746 1738340E FCF09CE3 88F666EB 38F7F3AF D42DC0B6 64D9F461 F31AA2ED C6B4AB71 BD42F4D7 C058E13F 64B237EF 7DDD2ABC 0DEB0C6C") +
        parse_hex("000025DE 37157F50 D75D320D D0682AB4 A67E4715 86FBC2D3 1AA32E69 57FA2B26 14C4CD40 A1E27283 EAAF4272 AE517847 197432E2 D61C85F5") * i,
        parse_hex("0001D407 B70B01E4 AEE172ED F491F4EF 32144F03 F5E054CE F9FDE5A3 5EFA3642 A1181790 5ED0D4F1 93F31124 264924A5 F64EFE14 B6EC97E5") +
        parse_hex("0000E7DE C8C32F50 A4E735A8 39DCDB89 FE0763A1 84C525F7 B7D0EBC0 E84E9D83 E9AC53A5 72A25D19 E1464B50 9D97272A E761657B 4765B3D6") * i)
P_B = E(parse_hex("00008664 865EA7D8 16F03B31 E223C26D 406A2C6C D0C3D667 466056AA E85895EC 37368BFC 009DFAFC B3D97E63 9F65E9E4 5F46573B 0637B7A9"),
        parse_hex("00006AE5 15593E73 97609197 8DFBD70B DA0DD6BC AEEBFDD4 FB1E748D DD9ED3FD CF679726 C67A3B2C C12B3980 5B32B612 E058A428 0764443B"))
Q_B = E(parse_hex("00012E84 D7652558 E694BF84 C1FBDAAF 99B83B42 66C32EC6 5B10457B CAF94C63 EB063681 E8B1E739 8C0B241C 19B9665F DB9E1406 DA3D3846"),
        parse_hex("0000EBAA A6C73127 1673BEEC E467FD5E D9CC29AB 564BDED7 BDEAA86D D1E0FDDF 399EDCC9 B49C829E F53C7D7A 35C3A074 5D73C424 FB4A5FD2") * i)

한 가지 불분명한 것은 문제에 $E_A$라고 나온 $\mathbb{F}_{p^2}$의 원소이다. SIDH와 SIKE에서는 Montgomery curve를 주로 다루기 때문에 이 값이 $y^2 = x^3 + Ax^2 + x$에서 $A$일 것 같은 킹리적 갓심이 든다. 한번 그렇게 해석해보자.

In [4]:
A_alice = parse_hex("0000C2D29711365E 5AC6CB621574EC6D 2EFAD33760BD5DB3 B74E01533E6E978B D2A992929C0A36D7 32B6AE51D0397225 B6FAC262931F77CD") + \
          parse_hex("00003CAABA21E68C 10C9ADD816980B1D 73822C2CC8D18BCE 22B0904CA99C1A57 8D39CA029E7D28E2 55F2A19941112230 89832831644D66E6") * i
E_A = EllipticCurve(k, [0, A_alice, 0, 1, 0])
phi_A_x_P_B = parse_hex("000133617DF3EEDE E09DA13528F184DF E8BD912929AC949A F8A887EE9B3A3E43 C6035F5B88E2D82A 2D5AD3C7B1243578 F33CC6C74346FB64") + \
              parse_hex("00000AA8F2D61712 05D359582EEFE035 5E9BA5DC169B0B66 7FCA9EBFF266C138 D7C174B258A561CE FC33AB3DC20AF224 4E4C0FD338EE4C84") * i
phi_A_y_P_B = sqrt(phi_A_x_P_B^3 + A_alice * phi_A_x_P_B^2 + phi_A_x_P_B)
phi_A_P_B = E_A(phi_A_x_P_B, phi_A_y_P_B)
phi_A_x_Q_B = parse_hex("0001A646437477E6 1DFD02DA472AC83C B439354D9F1F7879 9AFFEECD96028D6B E82E73C364291362 AB1F800A9DC4BFF5 58AABED07AE90E39") + \
              parse_hex("0000547960A0A8BA DE1E7B67DC14C850 81150853FEFD48BD 4B3F4F304B29D680 C7D2827C402A222C DB006F3FC8D7A13C 30B9DB783E6B463B") * i
phi_A_y_Q_B = sqrt(phi_A_x_Q_B^3 + A_alice * phi_A_x_Q_B^2 + phi_A_x_Q_B)
phi_A_Q_B = E_A(phi_A_x_Q_B, phi_A_y_Q_B)

SIKE 표준에 따라, curve point에서 elliptic curve를 얻는 함수 `cfpk`를 구현하면 다음과 같다.

In [5]:
def cfpk(x_P, x_Q, x_R):
    assert x_P != 0 and x_Q != 0 and x_R != 0
    A = (1 - x_P * x_Q - x_P * x_R - x_Q * x_R)^2 / (4 * x_P * x_Q * x_R) - x_P - x_Q - x_R
    return EllipticCurve(k, [0, A, 0, 1, 0])

이제 킹리적 갓심을 검증해보면 맞는 것으로 나온다.

In [6]:
cfpk(phi_A_x_P_B, phi_A_x_Q_B, (phi_A_P_B - phi_A_Q_B)[0]) == E_A

True